In [1]:
import geopandas as gpd
import numpy as np
import osmnx as ox
import pandas as pd
import yaml
from shapely import Point, Polygon

ox.settings.use_cache = True
ox.settings.log_console = False

with open("../data/crs.yaml", "r") as fp:
    crs = yaml.safe_load(fp)

In [2]:
CITY = "madrid"
# possible values: Eigenvector Centrality, Degree Centrality, Closeness Centrality, Betweenness Centrality
CENTRALITY = "Betweenness Centrality"

In [3]:
stops = pd.read_csv(f"../data/stops/{CITY}/stops_with_centrality.csv", engine="pyarrow")
stops["geometry"] = stops.apply(lambda x: Point(x["stop_lon"], x["stop_lat"]), axis=1)
stops = gpd.GeoDataFrame(stops, geometry="geometry", crs=4326)
stops.to_crs(crs[CITY], inplace=True)
stops.head(3)

,Node,Eigenvector Centrality,Degree Centrality,Closeness Centrality,Betweenness Centrality,stop_id,clust,stop_lat,stop_lon,stop_name,geometry
0,3.0,0.000003,0.001158,0.061367,0.006125,5130,3,40.50730,-3.68606,Monasterio De Arlanza - Nuestra Señora De Valv...,POINT (600111.334 656263.682)
1,3.0,0.000003,0.001158,0.061367,0.006125,5166,3,40.50767,-3.68597,Monasterio De Arlanza - Nuestra Señora De Valv...,POINT (600118.953 656304.721)
2,690.0,0.000014,0.002703,0.065333,0.016070,3881,690,40.51320,-3.67918,Carretera De Alcobendas - Bercianos,POINT (600693.765 656918.117)


In [4]:
def read_boundary(city: str) -> Polygon:
    boundary = gpd.read_file(f"../output/{city}/boundary.geojson").set_crs(4326)
    return boundary.geometry[0]


def determine_city_centroid_by_landuse(boundary: Polygon, crs: int) -> Point:
    landuse = ox.features_from_polygon(
        boundary,
        tags={
            "landuse": [
                "residential",
                # "retail",
                # "industrial",
            ],
        },
    )
    landuse = landuse[landuse["geometry"].geom_type == "Polygon"].copy()
    landuse.to_crs(crs, inplace=True)
    return landuse.union_all().centroid

In [5]:
landuse_centroid = determine_city_centroid_by_landuse(read_boundary(CITY), crs[CITY])
# gpd.GeoDataFrame(geometry=[landuse_centroid], crs=crs[CITY]).to_crs(4326)

In [6]:
# stops[stops["Betweenness Centrality"] > 0.3446914]

In [7]:
# based on: https://stackoverflow.com/a/38022636/4737417
def distance_matrix(data: pd.DataFrame):
    result = []
    # result.append(data["stop_id"].tolist())
    for i in data.itertuples():
        r = []
        for j in data.itertuples():
            d = np.round(i.geometry.distance(j.geometry))
            r.append(d)
        # result.append([i.stop_id] + r)
        result.append(r)
    result = np.array(result)
    return result


def medoid_index(distamce_matrix: np.array) -> int:
    return int(np.argmin(distamce_matrix.sum(axis=0)))


def calculate_medoid(data: pd.DataFrame, id_column: str = "stop_id") -> int:
    dmx = distance_matrix(data)
    i = int(np.argmin(dmx.sum(axis=0)))
    return data[id_column].tolist()[i]

In [8]:
# dmx = distance_matrix(stops[stops["Betweenness Centrality"] > 0.3446914])
# medoid_index(dmx)

In [9]:
maxc = stops[CENTRALITY].max()
medoid_id = calculate_medoid(stops[stops["Betweenness Centrality"] == maxc])
medoid = stops[stops["stop_id"] == medoid_id]["geometry"].tolist()[0]
centroid = stops[stops["Betweenness Centrality"] == maxc].union_all().centroid

In [10]:
# stops["distance_from_city_centroid"] = stops["geometry"].apply(
#     lambda x: np.round(x.distance(boundary.centroid) / 1000, 3)
# )

In [11]:
stops["distance_from_largest_betweenness_medoid"] = stops["geometry"].apply(
    lambda x: np.round(x.distance(medoid) / 1000, 3)
)
stops["distance_from_largest_betweenness_centroid"] = stops["geometry"].apply(
    lambda x: np.round(x.distance(centroid) / 1000, 3)
)
stops["distance_from_landuse_centroid"] = stops["geometry"].apply(
    lambda x: np.round(x.distance(landuse_centroid) / 1000, 3)
)

In [12]:
m = stops[
    [
        "stop_id",
        # "distance_from_city_centroid",
        "distance_from_largest_betweenness_centroid",
        "distance_from_largest_betweenness_medoid",
        "distance_from_landuse_centroid",
    ]
]
m.dropna(subset=["stop_id"]).to_csv(f"../output/{CITY}/distance.csv", index=False)